
# Generage figres from Micro-C data.
- This script uses the coolbox api along with data generates with micro-c, cooler, mustache, and juicer to plot regions of the Drosophila genome with differences in chromatin.  
### To create the conda environment for this script:

1. mamba create env -n coolbox -c bioconda coolbox 
2. mamba activate coolbox
3. mamba install -c anaconda ipykernel
4. python -m ipykernel install --user --name=coolbox

Then select the coolbox kernel under python environments within the notebook.
- https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084

### Import packages

In [1]:
import os
import coolbox
from coolbox.api import *
import pandas as pd
from math import floor

# change working directory
os.chdir("/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map")
print(f"Current working directory: {os.path.abspath(os.curdir)}")


Current working directory: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map


In [2]:
pairwise={
    'JW18-DOX-1_JW18-DOX-2':	['JW18-DOX-1',	'JW18-DOX-2'],
    'JW18-DOX-1_JW18-wMel-1':	['JW18-DOX-1',	'JW18-wMel-1'],
    'JW18-DOX-1_JW18-wMel-2':	['JW18-DOX-1',	'JW18-wMel-2'],
    'JW18-DOX-2_JW18-wMel-1':	['JW18-DOX-2',	'JW18-wMel-1'],
    'JW18-DOX-2_JW18-wMel-2':	['JW18-DOX-2',	'JW18-wMel-2'],
    'JW18-DOX-1_JW18-wRi-1':	['JW18-DOX-1',	'JW18-wRi-1'],
    'JW18-DOX-1_JW18-wRi-2':    ['JW18-DOX-1',	'JW18-wRi-2'],
    'JW18-DOX-2_JW18-wRi-1':	['JW18-DOX-2',	'JW18-wRi-1'],
    'JW18-DOX-2_JW18-wRi-2':	['JW18-DOX-2',	'JW18-wRi-2'],
    'JW18-DOX-1_JW18-wWil-1':	['JW18-DOX-1',	'JW18-wWil-1'],
    'JW18-DOX-1_JW18-wWil-2':	['JW18-DOX-1',	'JW18-wWil-2'],
    'JW18-DOX-2_JW18-wWil-1':	['JW18-DOX-2',	'JW18-wWil-1'],
    'JW18-DOX-2_JW18-wWil-2':	['JW18-DOX-2',	'JW18-wWil-2'],
    'JW18-wMel-1_JW18-wRi-1':	['JW18-wMel-1',	'JW18-wRi-1'],
    'JW18-wMel-1_JW18-wRi-2':	['JW18-wMel-1',	'JW18-wRi-2'],
    'JW18-wMel-2_JW18-wRi-1':	['JW18-wMel-2',	'JW18-wRi-1'],
    'JW18-wMel-2_JW18-wRi-2':	['JW18-wMel-2',	'JW18-wRi-2'],
    'JW18-wMel-1_JW18-wWil-1':	['JW18-wMel-1',	'JW18-wWil-1'],
    'JW18-wMel-1_JW18-wWil-2':	['JW18-wMel-1',	'JW18-wWil-2'],
    'JW18-wMel-2_JW18-wWil-1':	['JW18-wMel-2',	'JW18-wWil-1'],
    'JW18-wMel-2_JW18-wWil-2':	['JW18-wMel-2',	'JW18-wWil-2']
}


In [3]:
def plot(TEST_RANGE,pairwise,comparison,resolution,directory):
    #Contact map 1
    file1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][0]}.matrix_1kb.mcool'
    #Contact map2
    file2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][1]}.matrix_1kb.mcool'
    #genes with open regions that are enriched in file1
    gtf1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop1.symbol.gtf'
    #genes with open regions that are enriched in file1
    gtf2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop2.symbol.gtf'
    #mustache identified de loops1
    loops1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop1.bed'
    #mustache identified de loops2
    loops2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop2.bed'
    
    c1,c2=pairwise[comparison]

    with MinValue(-9), MaxValue(-1):
        cool1 = Cool(file1)
        cool2 = Cool(file2)

    frame = ChromName()+ XAxis() + \
        Selfish(cool2, cool1, norm="log") +Title("FDR Corrected\np-value")+ \
        MinValue(0.000000001) + MaxValue(0.1)  + \
        HiCDiff(cool2, cool1, normalize='log', diff_method='diff', cmap="bwr_r") + Title("Differential\nChromatin\nInteractions") + \
        MinValue(-6) + MaxValue(6)  + \
        TADCoverage(loops1, border_only=False, alpha=0.25, color='red',border_color='red') +\
        TADCoverage(loops2, border_only=False, alpha=0.25, color='blue',border_color='blue') +\
        GTF(gtf1)+Title(f"Genetic elements\nenriched in\n{c1}")+ \
        GTF(gtf2)+Title(f"Genetic elements\nenriched in\n{c2}")+ \
        XAxis()
    frame *= Feature(depth_ratio=0.35)
    frame.plot(TEST_RANGE)

    bsr = Browser(frame, reference_genome='/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/reference/dmel-all-chromosome_only-r6.46.genome')
    bsr.goto(TEST_RANGE)
    bsr.save(f'{directory}/{comparison}_{TEST_RANGE}.pdf')



In [4]:
# Browser for given ranges:
comparison = 'JW18-DOX-1_JW18-wMel-1' 
resolution = '10kb'
TEST_RANGE = '3L:10000-1000000'


file1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][0]}.matrix_1kb.mcool'
    #Contact map2
file2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][1]}.matrix_1kb.mcool'
#genes with open regions that are enriched in file1
gtf1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop1.symbol.gtf'
#genes with open regions that are enriched in file1
gtf2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop2.symbol.gtf'
#mustache identified de loops1
loops1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop1.bed'
#mustache identified de loops2
loops2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop2.bed'

c1,c2=pairwise[comparison]

with MinValue(-9), MaxValue(-1):
    cool1 = Cool(file1)
    cool2 = Cool(file2)

frame = ChromName()+ XAxis() + \
    Selfish(cool2, cool1, norm="log") +Title("FDR Corrected\np-value")+ \
    MinValue(0.000000001) + MaxValue(0.1)  + \
    HiCDiff(cool2, cool1, normalize='log', diff_method='diff', cmap="bwr_r") + Title("Differential\nChromatin\nInteractions") + \
    MinValue(-6) + MaxValue(6)  + \
    TADCoverage(loops1, border_only=False, alpha=0.25, color='red',border_color='red') +\
    TADCoverage(loops2, border_only=False, alpha=0.25, color='blue',border_color='blue') +\
    GTF(gtf1)+Title(f"Genetic elements\nenriched in\n{c1}")+ \
    GTF(gtf2)+Title(f"Genetic elements\nenriched in\n{c2}")+ \
    XAxis()
frame *= Feature(depth_ratio=0.35)
frame.plot(TEST_RANGE)

bsr = Browser(frame, reference_genome='/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/reference/dmel-all-chromosome_only-r6.46.genome')
bsr.goto(TEST_RANGE)
# bsr.save(f'{directory}/{comparison}_{TEST_RANGE}.pdf')

[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/10kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/10kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[WARNING:plot.py:486 -            plot_tads()] No regions found for Coverage TAD.1.
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/10kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop2.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/10kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/w

In [5]:
from matplotlib.colors import LinearSegmentedColormap

# # give a list of color (color gradient)
# uberplant = '#171123'
# eggplants = '#372248'
# periptant = '#414770'
# blueplant = '#5B85AA'
# kindablue = '#A9BCD0'
fullwhite = '#FFFFFF'
# sadorange = '#F9C784'
# tangerine = '#FCAF58'
# prforange = '#FF8C42'
# xxxorange = '#F46036'
# rustyredy = '#C2320A'

# diff_gradient = [uberplant, eggplants, periptant, blueplant, kindablue, fullwhite, sadorange, tangerine, prforange, xxxorange, rustyredy]
# selfish_gradient = [uberplant, eggplants, periptant, blueplant, kindablue, fullwhite]

purpl = '#241e4e'
blue1 = '#587792'
blue2 = '#8db1ab'
green = '#C7E576'
yell1 = '#f2ed6f'
yell2 = '#f4e04d'
yell3 = '#F0D719'
orang = '#ff4000'

diff_gradient = [purpl,blue1, blue2, green, fullwhite, yell1, yell2, yell3, orang]
selfish_gradient =[purpl, fullwhite]

diff_cmap = LinearSegmentedColormap.from_list('interaction', diff_gradient)
diff_cmap.set_bad(orang)
diff_cmap.set_under(purpl)


selfish_cmap = LinearSegmentedColormap.from_list('interaction', selfish_gradient)
selfish_cmap.set_bad(orang)
selfish_cmap.set_under(purpl)


In [6]:
# Browser for given ranges:
directory = '/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation'
comparison = 'JW18-DOX-1_JW18-wMel-1' 
resolution = '1kb'
TEST_RANGE = '3L:2980000-3970000'
# TEST_RANGE = '3R:31491078-31492678' 
JW18_DOX  = '#BAD86E'
JW18_wMel = purpl
#contact map 1
file1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][0]}.matrix_1kb.mcool'
#Contact map2
file2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][1]}.matrix_1kb.mcool'
#genes with open regions that are enriched in file1
gtf1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop1.symbol.gtf'
#genes with open regions that are enriched in file1
gtf1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop1.symbol.gtf'
#genes with open regions that are enriched in file1
gtf2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop2.symbol.gtf'
#mustache identified de loops1
loops1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop1.bed'
#mustache identified de loops2
loops2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop2.bed'


###Tracks
#RNA-coverage 
rnaseq_cov_1 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/RNA-seq_coverage/JW18DOX221117.avg.coverage.bw'
rnaseq_cov_2 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/RNA-seq_coverage/JW18wMel221117.avg.coverage.bw'
#Micro-C_coverage 
microc_cov_1 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/Micro-C_coverage/JW18-DOX-1.coverage.bw'
microc_cov_2 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/Micro-C_coverage/JW18-wMel-1.coverage.bw'
#Mappability 
mappability =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/dmel_6_genmap_e0_k150_final.bw'

c1,c2=pairwise[comparison]


with MinValue(-9), MaxValue(-1):
    cool1 = Cool(file1)
    cool2 = Cool(file2)

frame = ChromName()+ XAxis() + \
    Selfish(cool2, cool1, norm="log", cmap=selfish_cmap) +Title("FDR Corrected\np-value")+ \
    MinValue(0.000000001) + MaxValue(0.1)  + \
    HiCDiff(cool2, cool1, normalize='log', diff_method='diff', cmap=diff_cmap) + Title("Differential\nChromatin\nInteractions") + \
    MinValue(-6) + MaxValue(6)  + \
    TADCoverage(loops1, border_only=False, alpha=0.45, color=JW18_DOX,border_color=JW18_DOX) +\
    TADCoverage(loops2, border_only=False, alpha=0.25, color=JW18_wMel,border_color=JW18_wMel) +\
    GTF(gtf1)+Title(f"Genetic elements\nenriched in\n{c1}")+ \
    GTF(gtf2)+Title(f"Genetic elements\nenriched in\n{c2}")+ \
    BigWig(rnaseq_cov_1) + Color(JW18_DOX) + MinValue(0) + MaxValue(500)+ Title("RNA-seq coverage\n JW18-DOX-1")+\
    BigWig(rnaseq_cov_2) + Color(JW18_wMel) + MinValue(0) + MaxValue(500)+ Title("RNA-seq coverage\n JW18-wMel-1")+\
    BigWig(microc_cov_1) + Color(JW18_DOX) + MinValue(0) + MaxValue(5000)+ Title("Micro-C coverage\n JW18-DOX-1")+\
    BigWig(microc_cov_2) + Color(JW18_wMel) + MinValue(0) + MaxValue(5000)+ Title("Micro-C coverage\n JW18-wMel-1")+\
    BigWig(mappability) + Color('grey') + MinValue(0) + MaxValue(1)+ Title("Mappability")+TrackHeight(1) +\
    XAxis()
frame *= Feature(depth_ratio=0.35)
frame.plot(TEST_RANGE)

bsr = Browser(frame, reference_genome='/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/reference/dmel-all-chromosome_only-r6.46.genome')
bsr.goto(TEST_RANGE)
bsr.show()
bsr.save(f'{directory}/{comparison}_{TEST_RANGE}.pdf')

[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop2.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop2.bed.bgz.tbi


[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop2.bed.bgz.tbi


In [42]:
# Browser for given ranges:
directory = '/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation'
comparison = 'JW18-DOX-1_JW18-wMel-1' 
resolution = '1kb'
TEST_RANGE = '3L:2980000-3970000'
# TEST_RANGE = '3R:31491078-31492678' 
JW18_DOX  = '#BAD86E'
JW18_wMel = purpl
#contact map 1
file1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][0]}.matrix_1kb.mcool'
#Contact map2
file2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][1]}.matrix_1kb.mcool'
#genes with open regions that are enriched in file1
# gtf1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop1.symbol.gtf'
#all genes
gtf_all='/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/reference/dmel-all-r6.46.sorted.uniq.gtf'
# # #genes with open regions that are enriched in file1
# gtf1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop1.symbol.gtf'
# #genes with open regions that are enriched in file1
# gtf2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop2.symbol.gtf'
#Genes enriched in both 
gtf3=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop.symbol.gtf'


#mustache identified de loops1
loops1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop1.bed'
#mustache identified de loops2
loops2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop2.bed'

# print(f'cat {gtf1} {gtf2} |uniq > {gtf3}')

###Tracks
#RNA-coverage 
rnaseq_cov =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/RNA-seq_coverage/JW18wMel221117_vs_DOX.diff.coverage.bw'
# rnaseq_cov_1 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/RNA-seq_coverage/JW18DOX221117.avg.coverage.bw'
# rnaseq_cov_2 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/RNA-seq_coverage/JW18wMel221117.avg.coverage.bw'
#Micro-C_coverage 
microc_cov =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/Micro-C_coverage/JW18-wMel_vs_DOX.diff.coverage.bw'
# microc_cov_1 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/Micro-C_coverage/JW18-DOX-1.coverage.bw'
# microc_cov_2 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/Micro-C_coverage/JW18-wMel-1.coverage.bw'
#Mappability 
mappability =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/dmel_6_genmap_e0_k150_final.bw'

c1,c2=pairwise[comparison]

with MinValue(-9), MaxValue(-1):
    cool1 = Cool(file1)
    cool2 = Cool(file2)

frame = ChromName()+ XAxis() + \
    Selfish(cool2, cool1, norm="log", cmap=selfish_cmap) +Title("FDR Corrected\np-value")+ \
    MinValue(0.000000001) + MaxValue(0.1)  + \
    HiCDiff(cool2, cool1, normalize='log', diff_method='diff', cmap=diff_cmap) + Title("Differential\nChromatin\nInteractions") + \
    MinValue(-6) + MaxValue(6)  + \
    TADCoverage(loops1, border_only=False, alpha=0.45, color=JW18_DOX,border_color=JW18_DOX) +\
    TADCoverage(loops2, border_only=False, alpha=0.25, color=JW18_wMel,border_color=JW18_wMel) +\
    GTF(gtf1)+Title(f"Genetic elements\nenriched in\n{c1}")+ TrackHeight(3)+\
    GTF(gtf2)+Title(f"Genetic elements\nenriched in\n{c2}")+ TrackHeight(3)+\
    Spacer(height=.5) +\
    BigWig(rnaseq_cov) + Color(JW18_wMel) + MinValue(0) + MaxValue(500)+ Title("RNA-seq coverage\nJW18-wMel enrichment\nover JW18-DOX")+TrackHeight(1)+\
    BigWig(rnaseq_cov) + Color(JW18_DOX) + MinValue(-500) + MaxValue(0)+ TrackHeight(1)+\
    Spacer(height=.5) +\
    BigWig(microc_cov) + Color(JW18_wMel) + MinValue(0) + MaxValue(1000)+ Title("Micro-C coverage\nJW18-wMel enrichment\nover JW18-DOX")+TrackHeight(1)+\
    BigWig(microc_cov) + Color(JW18_DOX) + MinValue(-1000) + MaxValue(0)+ TrackHeight(1)+\
    Spacer(height=.5) +\
    BigWig(mappability) + MinValue(0) + MaxValue(1)+ Title("Mappability")+TrackHeight(1) +\
    XAxis()
frame *= Feature(depth_ratio=0.35)
frame.plot(TEST_RANGE)

bsr = Browser(frame, reference_genome='/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/reference/dmel-all-chromosome_only-r6.46.genome')
bsr.goto(TEST_RANGE)
bsr.show()
bsr.save(f'{directory}/{comparison}_{TEST_RANGE}.svg')
print(f"File saved to: {directory}/{comparison}_{TEST_RANGE}.svg")

[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop2.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop2.bed.bgz.tbi


[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop1.bed.bgz.tbi
[W::hts_idx_load3] The index file is older than the data file: /scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/1kb/p_value_0.0001/bed/plot_bed/JW18-DOX-1_JW18-wMel-1.diffloop2.bed.bgz.tbi


File saved to: /scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/JW18-DOX-1_JW18-wMel-1_3L:2980000-3970000.svg


In [21]:

# # TEST_RANGE = '3L:2980000-3970000'

# def plot_test_range(TEST_RANGE,folder):
#     # Browser for given ranges:
#     directory = f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/comparisons/plots/{folder}'
#     comparison = 'JW18-DOX-1_JW18-wMel-1' 
#     resolution = '1kb'

#     JW18_DOX  = '#BAD86E'
#     JW18_wMel = purpl
#     #contact map 1
#     file1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][0]}.matrix_1kb.mcool'
#     #Contact map2
#     file2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/{pairwise[comparison][1]}.matrix_1kb.mcool'
#     #all genes
#     gtf_all='/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/reference/dmel-all-r6.46.sorted.gtf'
#     #genes with open regions that are enriched in file1
#     gtf1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop1.symbol.gtf'
#     #genes with open regions that are enriched in file1
#     gtf2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/gtf/symbol/{comparison}.diffloop2.symbol.gtf'
#     #mustache identified de loops1
#     loops1=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop1.bed'
#     #mustache identified de loops2
#     loops2=f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop2.bed'


#     ###Tracks
#     #RNA-coverage 
#     rnaseq_cov_1 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/RNA-seq_coverage/JW18DOX221117.avg.coverage.bw'
#     rnaseq_cov_2 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/RNA-seq_coverage/JW18wMel221117.avg.coverage.bw'
#     #Micro-C_coverage 
#     microc_cov_1 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/Micro-C_coverage/JW18-DOX-1.coverage.bw'
#     microc_cov_2 =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/Micro-C_coverage/JW18-wMel-1.coverage.bw'
#     #Mappability 
#     mappability =f'/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/dmel_6_genmap_e0_k150_final.bw'

#     c1,c2=pairwise[comparison]

#     with MinValue(-9), MaxValue(-1):
#         cool1 = Cool(file1)
#         cool2 = Cool(file2)

#     frame = ChromName()+ XAxis() + \
#         Selfish(cool2, cool1, norm="log", cmap=selfish_cmap) +Title("FDR Corrected\np-value")+ \
#         MinValue(0.000000001) + MaxValue(0.1)  + \
#         HiCDiff(cool2, cool1, normalize='log', diff_method='diff', cmap=diff_cmap) + Title("Differential\nChromatin\nInteractions") + \
#         MinValue(-6) + MaxValue(6)  + \
#         TADCoverage(loops1, border_only=False, alpha=0.45, color=JW18_DOX,border_color=JW18_DOX) +\
#         TADCoverage(loops2, border_only=False, alpha=0.25, color=JW18_wMel,border_color=JW18_wMel) +\
#         GTF(gtf_all)+Title(f"DmeL GTF\n")+ \
#         GTF(gtf1)+Title(f"Genetic elements\nenriched in\n{c1}")+ \
#         GTF(gtf2)+Title(f"Genetic elements\nenriched in\n{c2}")+ \
#         BigWig(rnaseq_cov_1) + Color(JW18_DOX) + MinValue(0) + MaxValue(500)+ Title("RNA-seq coverage\n JW18-DOX-1")+\
#         BigWig(rnaseq_cov_2) + Color(JW18_wMel) + MinValue(0) + MaxValue(500)+ Title("RNA-seq coverage\n JW18-wMel-1")+\
#         BigWig(microc_cov_1) + Color(JW18_DOX) + MinValue(0) + MaxValue(5000)+ Title("Micro-C coverage\n JW18-DOX-1")+\
#         BigWig(microc_cov_2) + Color(JW18_wMel) + MinValue(0) + MaxValue(5000)+ Title("Micro-C coverage\n JW18-wMel-1")+\
#         BigWig(mappability) + Color('grey') + MinValue(0) + MaxValue(1)+ Title("Mappability")+TrackHeight(1) +\
#         XAxis()
#     frame *= Feature(depth_ratio=0.35)
#     frame.plot(TEST_RANGE)

#     bsr = Browser(frame, reference_genome='/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/reference/dmel-all-chromosome_only-r6.46.genome')
#     bsr.goto(TEST_RANGE)
#     # bsr.show()
#     bsr.save(f'{directory}/{comparison}_{TEST_RANGE}.pdf')

# import glob
# import os

# folder = '/scratch1/jodie/wolbachia/Wolbachia_induced_differentiation/comparisons'
# file_paths = glob.glob(f"{folder}/*.txt")

# # Extract filenames without path and extension
# file_names = [os.path.splitext(os.path.basename(path))[0] for path in file_paths]

# # Create folders based on these names
# for name in file_names:
#     # Define the directory path where you want to create the new folder
#     new_dir_path = os.path.join(folder,'plots', name)
#    # Create the directory (if it doesn't already exist)
#     os.makedirs(new_dir_path, exist_ok=True)

#     comparison = pd.read_csv(f'{folder}/{name}.txt', sep='\t')
#     for location in comparison['location']:
#         # print(new_dir_path)
#         plot_test_range(location,name)




In [9]:
# resolution='10kb'

# for comparison in pairwise.keys():
#     directory=f"/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/plots/{resolution}/{comparison}"
#     if not os.path.exists(directory):
#         # If the directory does not exist, create it
#         os.makedirs(directory)
#     ranges1=pd.read_csv(f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop1.bed', sep='\t', header=None)
#     ranges2=pd.read_csv(f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop2.bed', sep='\t', header=None)
#     ranges=pd.concat([ranges1,ranges2],axis=0)
#     ranges.drop_duplicates()

#     for index, row in ranges.iterrows():
#     # Print the first, second, and third elements of each row
#         TEST_RANGE=f"{row[0]}:{floor(row[1]/1.1)}-{floor(row[2]*1.1)}"
        
#         plot(TEST_RANGE,pairwise,comparison,resolution,directory)

In [10]:
# resolution='1kb'

# for comparison in pairwise.keys():
#     directory=f"/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/plots/{resolution}/{comparison}"
#     if not os.path.exists(directory):
#         # If the directory does not exist, create it
#         os.makedirs(directory)
#     ranges1=pd.read_csv(f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop1.bed', sep='\t', header=None)
#     ranges2=pd.read_csv(f'/scratch1/jodie/wolbachia/Micro-C/09Nov2023_Micro_C/map/mustache_v1/{resolution}/p_value_0.0001/bed/plot_bed/{comparison}.diffloop2.bed', sep='\t', header=None)
#     ranges=pd.concat([ranges1,ranges2],axis=0)
#     ranges.drop_duplicates()

#     for index, row in ranges.iterrows():
#     # Print the first, second, and third elements of each row
#         TEST_RANGE=f"{row[0]}:{floor(row[1]/1.1)}-{floor(row[2]*1.1)}"
        
#         plot(TEST_RANGE,pairwise,comparison,resolution,directory)